In [ ]:
!pip install pmdarima

import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Membaca data
df = pd.read_csv('/content/Data_APBN.csv', index_col='Waktu', parse_dates=True)
df = df.dropna()
print('Shape of data:', df.shape)
print(df.head())

# Tampilkan daftar kolom yang tersedia
print("Kolom yang tersedia untuk dianalisis:")
print(list(df.columns))

# Input dari pengguna untuk memilih kolom yang dianalisis
kolom_dipilih = input("Masukkan nama kolom yang ingin dianalisis: ").strip()

# Validasi input kolom
if kolom_dipilih not in df.columns:
    raise ValueError(f"Kolom '{kolom_dipilih}' tidak ditemukan dalam dataset. Pilih kolom yang benar.")

# Visualisasi kolom yang dipilih
print(f"Visualisasi kolom '{kolom_dipilih}':")
df[kolom_dipilih].plot(figsize=(12, 5), title=f"{kolom_dipilih} Over Time")
plt.show()

# Fungsi untuk uji stasioneritas
def ad_test(dataset):
    print("\n### Uji Stasioneritas ADF ###")
    dftest = adfuller(dataset, autolag='AIC')
    print("1. ADF : ", dftest[0])
    print("2. P-Value : ", dftest[1])
    print("3. Num Of Lags : ", dftest[2])
    print("4. Num Of Observations Used For ADF Regression:", dftest[3])
    print("5. Critical Values :")
    for key, val in dftest[4].items():
        print("\t", key, ": ", val)

# Uji stasioneritas
ad_test(df[kolom_dipilih])

# Mencari order ARIMA terbaik
print("\n### Mencari Order ARIMA Terbaik ###")
stepwise_fit = auto_arima(df[kolom_dipilih], suppress_warnings=True)
print(stepwise_fit.summary())

# Pembagian data train dan test (80% train, 20% test)
train_size = int(len(df) * 0.8)
train = df.iloc[:train_size]
test = df.iloc[train_size:]

# Membuat model ARIMA dengan order terbaik
print("\n### Training Model ARIMA ###")
order = stepwise_fit.order
model = ARIMA(train[kolom_dipilih], order=order)
model = model.fit()
print(model.summary())

# Prediksi pada data test
start = len(train)
end = len(df) - 1
pred = model.predict(start=start, end=end, typ='levels')

# Evaluasi model: RMSE, MSE, MAE
rmse = sqrt(mean_squared_error(pred, test[kolom_dipilih]))
mae = mean_absolute_error(pred, test[kolom_dipilih])
mse = mean_squared_error(pred, test[kolom_dipilih])

rmse_persen = (rmse / rata_rata_aktual) * 100
mae_persen = (mae / rata_rata_aktual) * 100
mse_persen = (mse / (rata_rata_aktual**2)) * 100  # MSE dibagi kuadrat rata-rata agar proporsional

print("\n### Evaluasi Model ###")
print("RMSE (Root Mean Squared Error):", rmse)
print("MAE (Mean Absolute Error):", mae)
print("MSE (Mean Squared Error):", mse)
print("RMSE (%):", rmse_persen, "%")
print("MAE (%):", mae_persen, "%")
print("MSE (%):", mse_persen, "%")


# Prediksi 30 periode ke depan
index_future_dates = pd.date_range(start=df.index[-1], periods=30, freq='M')
pred_future = model.predict(start=len(df), end=len(df) + 29, typ='levels').rename('ARIMA Predictions')
pred_future.index = index_future_dates
print(pred_future)
index_future_dates = pd.date_range(start=df.index[-1], periods=30, freq='M')  
plt.figure(figsize=(12, 5))
plt.plot(df[kolom_dipilih], label='Data Aktual')  # Plot data aktual
# Convert the index of pred_future to a NumPy array of datetime objects
# This should help Matplotlib handle the dates correctly
plt.title('Prediksi 30 Periode ke Depan')
plt.xlabel('Waktu')
plt.show() # Added show() to display plot

# Perbandingan Prediksi vs Aktual pada data test
print("\n### Perbandingan Prediksi vs Data Aktual ###")
pred.index = test.index
comparison_df = pd.DataFrame({'Actual': test[kolom_dipilih], 'Predictions': pred})
comparison_df.plot(legend=True, figsize=(12, 5), title="Perbandingan Aktual vs Prediksi")
plt.show()
